Discutons des concepts de base de l'architecture des GANs.

# Architecture de GANS
Un GAN se compose de deux réseaux indépendants, un Générateur et un Discriminateur.

Le Générateur génère des échantillons synthétiques à partir d'un bruit aléatoire (échantillonné à partir d'un espace latent) et le Discriminateur est un classificateur binaire qui distingue si l'échantillon d'entrée est réel (renvoie une valeur scalaire 1) ou faux (renvoie une valeur scalaire 0).

<p align="center">
  <img src="images/fig.1-Generator-and-Discriminator.jpg" alt="Fig1: Générateur et Discriminateur en tant que blocs de construction de GAN">
</p>

Les échantillons générés par le Générateur sont appelés des échantillons faux. Lorsqu'un point de données du jeu de données d'entraînement est donné en entrée au Discriminateur, il le considère comme un échantillon réel, tandis qu'il considère l'autre point de données comme faux lorsqu'il est généré par le Générateur.

Le Discriminateur veut faire son travail de la meilleure manière possible. Lorsqu'un échantillon faux (généré par le Générateur) est donné au Discriminateur, il veut le considérer comme faux, mais le Générateur veut générer des échantillons de manière à ce que le Discriminateur fasse une erreur en le considérant comme réel. En quelque sorte, le Générateur essaie de tromper le Discriminateur.

<p align="center">
  <img src="images/fig2-Generator-and-Discriminator.jpg" alt="Fig2: Générateur et Discriminateur en tant que blocs de construction de GAN">
</p>

La beauté de cette formulation réside dans la nature adversariale entre le Générateur et le Discriminateur.


## Loss Function 

Jetons un coup d'œil rapide à la fonction objectif et à la manière dont l'optimisation est effectuée. C'est une formulation d'optimisation min-max où le Générateur veut minimiser la fonction objectif tandis que le Discriminateur veut maximiser la même fonction objectif.

La premiére figure en bas en bas illustre la fonction objectif en cours d'optimisation. La fonction du Discriminateur est désignée par $D$ et la fonction du Générateur par $G$. $P_z$ est la distribution de probabilité de l'espace latent, qui est généralement une distribution gaussienne aléatoire. $P_{data}$ est la distribution de probabilité du jeu de données d'entraînement. Lorsque $x$ est échantillonné à partir de $P_{data}$, le Discriminateur veut le classer comme un échantillon réel. $G(z)$ est un échantillon généré ; lorsque $G(z)$ est donné en entrée au Discriminateur, il veut le classer comme un échantillon faux.

<p align="center">
  <img src="images/fig3-Objective-function-descriminative-perspective.jpg" alt="Fig3: Fonction objectif dans la formulation des GAN">
</p>

Le Discriminateur veut amener la probabilité de $D(G(z))$ à 0. Par conséquent, il veut maximiser $(1-D(G(z)))$, tandis que le Générateur veut forcer la probabilité de $D(G(z))$ à 1 afin que le Discriminateur fasse une erreur en considérant un échantillon généré comme réel. Par conséquent, le Générateur veut minimiser $(1-D(G(z)))$.

<p align="center">
  <img src="images/fig4.-Objective-function-generative-perspective.jpg" alt="Fig4: Fonction objectif dans la formulation des GAN">
</p>


## Implémentation Pytorch 

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils as utils
import torchvision

In [8]:
# Set a random seed for reproducibility
torch.manual_seed(0)

# Determine if a GPU is available and set the device accordingly
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Available_device: {device}')

Available_device: cuda


In [2]:
# Load and prepare the MNIST dataset
data = utils.data.DataLoader(
    torchvision.datasets.MNIST('./data', 
                               transform=torchvision.transforms.ToTensor(), 
                               download=True),
    batch_size=128,
    shuffle=True)

**Implémentation du Discriminateur**

L'architecture du discriminateur ici comporte trois couches cachées et utilise la technique de dropout pour la régularisation. Elle utilise la fonction d'activation Leaky ReLU pour introduire la non-linéarité et la fonction Sigmoïde pour produire une probabilité. Cette configuration est essentielle pour le processus d'apprentissage adversarial dans les GANs, où le Discriminateur apprend à distinguer les images réelles des images générées.

In [3]:
class Discriminator(nn.Module):
    """
    A three hidden-layer discriminative neural network.
    The Discriminator classifies input images as real or fake.
    """
    def __init__(self):
        super(Discriminator, self).__init__()
        self.linear1 = nn.Linear(784, 1024)
        self.linear2 = nn.Linear(1024, 512)
        self.linear3 = nn.Linear(512, 256)
        self.out = nn.Linear(256, 1)

    def forward(self, x):
        x = torch.flatten(x, start_dim=1)
        
        x = F.dropout(F.leaky_relu(self.linear1(x), negative_slope=0.2), p=0.3)
        x = F.dropout(F.leaky_relu(self.linear2(x), negative_slope=0.2), p=0.3)
        x = F.dropout(F.leaky_relu(self.linear3(x), negative_slope=0.2), p=0.3)
        
        return torch.sigmoid(self.out(x))

**Implémentation du Générateur**

L'architecture du générateur ici comporte trois couches cachées. Le Générateur crée des images synthétiques à partir de bruit aléatoire. Il utilise la fonction d'activation Leaky ReLU pour introduire la non-linéarité et la fonction Tanh pour normaliser les valeurs des pixels des images générées dans la plage [-1, 1]. Cette configuration est essentielle pour le processus d'apprentissage adversarial dans les GANs, où le Générateur apprend à produire des images réalistes capables de tromper le Discriminateur.

In [4]:
class Generator(nn.Module):
    """
    A three hidden-layer generative neural network.
    The Generator creates synthetic images from random noise.
    """
    def __init__(self):
        super(Generator, self).__init__()
        self.linear1 = nn.Linear(100, 256)
        self.linear2 = nn.Linear(256, 512)
        self.linear3 = nn.Linear(512, 1024)
        self.out = nn.Linear(1024, 784)

    def forward(self, x):
        x = F.leaky_relu(self.linear1(x), negative_slope=0.2)
        x = F.leaky_relu(self.linear2(x), negative_slope=0.2)
        x = F.leaky_relu(self.linear3(x), negative_slope=0.2)
        x = torch.tanh(self.out(x))
        return x

**Loss Function et optimisateur**

Nous utilisons l'optimiseur Adam pour les deux modèles. L'optimiseur Adam est choisi pour sa capacité à adapter les taux d'apprentissage pour chaque paramètre individuellement, ce qui peut conduire à une convergence plus rapide et plus stable. Les taux d'apprentissage pour les deux optimiseurs sont fixés à 0.0002.

Cette configuration permet d'entraîner efficacement les deux réseaux de neurones (Générateur et Discriminateur) de manière adversariale, chacun améliorant ses performances en réponse aux modifications de l'autre.

In [9]:
# Initialize the models
generator = Generator()
discriminator = Discriminator()

# Loss function and optimizers
criterion = nn.BCELoss()
optimizer_g = torch.optim.Adam(generator.parameters(), lr=0.0002)
optimizer_d = torch.optim.Adam(discriminator.parameters(), lr=0.0002)

**Entrainement**

L'entraînement du GAN commence par la configuration du logger pour suivre et visualiser les métriques d'entraînement. 

Une boucle d'entraînement est exécutée pendant 50 epcohes. À chaque époque, pour chaque lot d'échantillons réels de la base de données MNIST, le Discriminateur est d'abord entraîné à distinguer entre les échantillons réels et les échantillons générés par le Générateur à partir de bruit aléatoire. Ensuite, le Générateur est entraîné à améliorer sa capacité à tromper le Discriminateur en générant des échantillons plus réalistes. 

Les pertes pour le Discriminateur et le Générateur sont calculées et utilisées pour mettre à jour les poids des modèles via rétropropagation. Le logger enregistre les pertes et, à intervalles réguliers, affiche l'état de l'entraînement et enregistre des images générées pour visualisation. Enfin, les paramètres des modèles sont sauvegardés à la fin de chaque époque, et le logger est fermé après la fin de l'entraînement.

In [6]:
# Logger setup
from helpers import Logger
logger = Logger(model_name='GAN', data_name='MNIST')

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    for n, (real_samples, _) in enumerate(data):
        # Data for training the discriminator
        real_samples = real_samples.view(-1, 784)
        real_samples_labels = torch.ones((real_samples.size(0), 1))
        latent_space_samples = torch.randn((real_samples.size(0), 100))
        generated_samples = generator(latent_space_samples)
        generated_samples_labels = torch.zeros((real_samples.size(0), 1))

        # Concatenate real and fake data
        all_samples = torch.cat((real_samples, generated_samples))
        all_samples_labels = torch.cat((real_samples_labels, generated_samples_labels))

        # Training the discriminator
        discriminator.zero_grad()
        output_d = discriminator(all_samples)
        loss_d = criterion(output_d, all_samples_labels)
        loss_d.backward()
        optimizer_d.step()

        # Data for training the generator
        latent_space_samples = torch.randn((real_samples.size(0), 100))
        generator.zero_grad()
        generated_samples = generator(latent_space_samples)
        output_d_generated = discriminator(generated_samples)

        # Reverse the labels for the generator
        generated_samples_labels = torch.ones((real_samples.size(0), 1))

        # Training the generator
        loss_g = criterion(output_d_generated, generated_samples_labels)
        loss_g.backward()
        optimizer_g.step()

        # Logging
        logger.log(d_error=loss_d, g_error=loss_g, epoch=epoch, n_batch=n, num_batches=len(data))
        if n % 100 == 0:
            logger.display_status(epoch, num_epochs, n, len(data), loss_d, loss_g, discriminator(real_samples), output_d_generated)

        if n % 100 == 0:
            logger.log_images(generated_samples.view(real_samples.size(0), 1, 28, 28), num_images=real_samples.size(0), epoch=epoch, n_batch=n, num_batches=len(data))

    # Save the model parameters
    logger.save_models(generator, discriminator, epoch)

# Close the logger
logger.close()

print("Training Finished.")

Epoch: [0/50], Batch Num: [0/469]
Discriminator Loss: 0.6952, Generator Loss: 0.7206
D(x): 0.5031, D(G(z)): 0.4864
Epoch: [0/50], Batch Num: [100/469]
Discriminator Loss: 1.4112, Generator Loss: 0.9719
D(x): 0.2959, D(G(z)): 0.4288
Epoch: [0/50], Batch Num: [200/469]
Discriminator Loss: 0.7556, Generator Loss: 0.7855
D(x): 0.4524, D(G(z)): 0.4602
Epoch: [0/50], Batch Num: [300/469]
Discriminator Loss: 0.6967, Generator Loss: 0.4956
D(x): 0.7548, D(G(z)): 0.6123
Epoch: [0/50], Batch Num: [400/469]
Discriminator Loss: 0.5493, Generator Loss: 0.8256
D(x): 0.6083, D(G(z)): 0.4408
Epoch: [1/50], Batch Num: [0/469]
Discriminator Loss: 0.7208, Generator Loss: 0.8516
D(x): 0.4406, D(G(z)): 0.4289
Epoch: [1/50], Batch Num: [100/469]
Discriminator Loss: 0.5490, Generator Loss: 0.8161
D(x): 0.6960, D(G(z)): 0.4478
Epoch: [1/50], Batch Num: [200/469]
Discriminator Loss: 0.5659, Generator Loss: 3.1000
D(x): 0.4942, D(G(z)): 0.0596
Epoch: [1/50], Batch Num: [300/469]
Discriminator Loss: 0.5748, Gene